In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1 # MTCNN para identificar la cara formato ya lo trae entrenado en el modelo
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os
#Proyecto que reconozca e identifique que va a ser la cara de alex etc. 

# Inicializar MTCNN y InceptionResnetV1

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40)
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) #reconocer de quien es la cara . En la carpeta poner le nombre de la persona. Al encender la camara decir que es el y por otra parte en que parte esta la caradentro dela 
resnet = InceptionResnetV1(pretrained='vggface2').eval()  #Modelo Preentrenado

# Cargar las imagenes

dataset = datasets.ImageFolder('C:\\Users\\FUNDACION NATAN\\Desktop\\PENGUIN ACADEMY\\Penguin\\live_face_recognition-main\\photos') # Cargar las fotos contiene las iamgenes de las personas
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

name_list = [] # nombres correspondientes a las fotos
embedding_list = [] # lista conteniendo las representaciones matriciales usando la resnet es la esencia matematica de la cara de alex 
for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True) # probabilidad reconocer la cara de alex 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0))
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])
#name list fabri y marc
# Guardamos los datos generados para luego poder identificar las caras
data = [embedding_list, name_list]
torch.save(data, 'C:\\Users\\FUNDACION NATAN\\Desktop\\PENGUIN ACADEMY\\Penguin\\live_face_recognition-main\\photos\\data.pt')

# Cargar el modelo
load_data = torch.load('C:\\Users\\FUNDACION NATAN\\Desktop\\PENGUIN ACADEMY\\Penguin\\live_face_recognition-main\\photos\\data.pt')
embedding_list = load_data[0]
name_list = load_data[1]

#frame es fotograma pro segundo 
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        print("fail to grab frame, try again")
        break

    img = Image.fromarray(frame)

    # Utilizamos mtcnn para idenficar la CARA de la persona
    img_cropped_list, prob_list = mtcnn(img, return_prob=True)

    if img_cropped_list is not None:

        boxes, _ = mtcnn.detect(img)

        for i, prob in enumerate(prob_list):
            # si la probabilidad de que sea una cara es >90
            if prob>0.90:
                # Realizamos el embeding de la imagen capturada
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach()

                dist_list = []
                # Calculamos las distancias de todos los puntos del modelo con la imagen que estamos capturando
                # La pareja que menos distancia tenga es la persona reconocida

                for idx, emb_db in enumerate(embedding_list): # Comparamos 1 a 1 con nuestra base de datos
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # Cogemos el minimo valor (el que mas se parece)
                min_dist_idx = dist_list.index(min_dist) # Indice del menor valor
                name = name_list[min_dist_idx] # Nombre de la persona correspondiente al minimo valor

                box = boxes[i] # recuadro de la cara correspondiente

                # Guardamos una copia del frame procesado y añadimos recuadro reconocimiento facial
                # + distancia y Nombre de la persona

                original_frame = frame.copy()

                if min_dist<0.90:
                    frame = cv2.putText(frame, name+' '+str(min_dist), (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)

                frame = cv2.rectangle(frame, (box[0],box[1]) , (box[2],box[3]), (255,0,0), 2)

    # Mostramos la imagen con dibujo y Nombre
    cv2.imshow("IMG", frame)


    k = cv2.waitKey(1)
    if k%256==27:
        print('Esc pressed, closing...')
        #break

    elif k%256==32:
        print('Enter your name :')
        name = input()

        if not os.path.exists('/Users/MarcSanmillan/Desktop/Penguin /live_face_recognition-main/photos/'+name):
            os.mkdir('photos/'+name)

        img_name = "/Users/MarcSanmillan/Desktop/Penguin /live_face_recognition-main/photoss/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))


cam.release()
cv2.destroyAllWindows()
